In [9]:
import datetime
import json
import pandas as pd
import glob
import os
import numpy as np

today_str=str(datetime.datetime.now().date())

writer_floder="/home/jian/Projects/Big_Lots/Weather/Forecast_Validation/"+today_str+"/"
try:
    os.stat(writer_floder)
except:
    os.mkdir(writer_floder)

In [10]:
forecast_file_list=glob.glob("/home/jian/Projects/Big_Lots/Weather/Json_data/forecast/forcast_api_response_20180524/"+"*.json")

actual_file_list=glob.glob("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_20180524/"+"*.json")

In [11]:
forecast_file_Q1_df=pd.DataFrame()
actual_file_Q1_df=pd.DataFrame()
i=1
for file in forecast_file_list:
    location=file
    date=datetime.datetime.strptime(file.split("/")[len(file.split("/"))-1][0:10],"%Y-%m-%d").date()
    
    if (date>=datetime.date(2018,2,4)) & (date<=datetime.date(2018,5,5)) & (date.weekday()==1): #Tuesday only
        forecast_file_Q1_df=forecast_file_Q1_df.append(pd.DataFrame({"Date_Monday":date,'Date_Saturday':date+datetime.timedelta(days=4),"forecast_file":location},index=[i]))
        i=i+1
i=1
for file in actual_file_list:
    location=file
    date=datetime.datetime.strptime(file.split("/")[len(file.split("/"))-1][0:10],"%Y-%m-%d").date()
    
    if (date>=datetime.date(2018,2,4)) & (date<=datetime.date(2018,5,5)) & (date.weekday()==5): #Monday only
        actual_file_Q1_df=actual_file_Q1_df.append(pd.DataFrame({"Date_Saturday":date,"actual_file":location},index=[i]))
        i=i+1
both_file_Q1=pd.merge(actual_file_Q1_df,forecast_file_Q1_df,on="Date_Saturday",how="inner")

In [12]:
inclusion_stores=pd.read_excel("/home/jian/Projects/Big_Lots/Q1_Post/BL_Sales YoY_JL_20180618.xlsx",sheetname="Inclusion Stores",skiprows=1,dtype=str)
store_zips=pd.read_excel("/home/jian/Projects/Big_Lots/Other_Input/all_store_DMA.xlsx",dtype=str)[['location_id','zip']]
store_zips['zip']=store_zips['zip'].apply(lambda x: x.zfill(5))
inclusion_stores=pd.merge(inclusion_stores,store_zips,on="location_id",how="left")
inclusion_stores=inclusion_stores[['location_id','zip']].rename(columns={"zip":"zip_cd"})
inclusion_stores=inclusion_stores[inclusion_stores['location_id']!='1769']

In [13]:
all_forecast_group_value=[]
all_forecast_desc_value=[]

all_actual_group_value=[]
all_actual_desc_value=[]

for i in range(len(both_file_Q1)):
    json_actual=json.load(open(both_file_Q1['actual_file'][i],"r"))
    json_forecast=json.load(open(both_file_Q1['forecast_file'][i],"r"))
    for zip_cd in inclusion_stores['zip_cd'].unique().tolist():
        if (zip_cd in list(json_forecast.keys())) & (zip_cd in list(json_actual.keys())):
            weather_forecast_list=json_forecast[zip_cd]['list'][31]['weather']
            weather_actual_list=json_actual[zip_cd]['weather']

            for j in range(len(weather_forecast_list)):
                weather_forecast_group=weather_forecast_list[j]['main']
                weather_forecast_desc=weather_forecast_list[j]['description']
                all_forecast_group_value=list(set(all_forecast_group_value+[weather_forecast_group]))
                all_forecast_desc_value=list(set(all_forecast_desc_value+[weather_forecast_desc]))
            
            for j in range(len(weather_actual_list)):    
                weather_actual_group=weather_actual_list[j]['main']
                weather_actual_desc=weather_actual_list[j]['description']
                all_actual_group_value=list(set(all_actual_group_value+[weather_actual_group]))
                all_actual_desc_value=list(set(all_actual_desc_value+[weather_actual_desc]))
        else:
            print(zip_cd,both_file_Q1['Date_Saturday'][i])
            
        
        
        

47546 2018-02-24
30106 2018-02-24
37620 2018-02-24
76117 2018-02-24
30458 2018-02-24
31419 2018-02-24
44511 2018-02-24
34112 2018-02-24
27407 2018-02-24
94509 2018-02-24
92553 2018-02-24
89431 2018-02-24
95051 2018-02-24
76063 2018-02-24
76116 2018-02-24
39402 2018-02-24
27534 2018-02-24
72015 2018-02-24
33813 2018-02-24
13905 2018-02-24
35806 2018-04-28


In [14]:
all_forecast_group_value

['Clouds', 'Rain', 'Clear', 'Snow']

In [15]:
rain_snow_weight=pd.read_excel("/home/jian/Projects/Big_Lots/Weather/Q1_Weather_Counts/Q1_inclusion_store_all_weather_type_ranked.xlsx",sheetname="ranked rain snow")

rain_weight=rain_snow_weight.iloc[:,[4,6]]
rain_weight.columns=[['rain_desc','weight']]
rain_weight['weight']=rain_weight['weight'].astype(int)
rain_weight=rain_weight.sort_values("weight")

snow_weight=rain_snow_weight.iloc[:,[0,2]]
snow_weight.columns=[['snow_desc','weight']]
snow_weight=snow_weight[~pd.isnull(snow_weight['snow_desc'])]
snow_weight['weight']=snow_weight['weight'].astype(int)
snow_weight=snow_weight.sort_values("weight")

group_weight=pd.read_excel("/home/jian/Projects/Big_Lots/Weather/Q1_Weather_Counts/Q1_inclusion_store_all_weather_type_ranked.xlsx",sheetname="all_weather_group_list")
group_weight['Severity']=group_weight['Severity'].astype(int)
group_weight=group_weight[['all_type_group','Severity']]
group_weight_dict=group_weight[['all_type_group', 'Severity']].set_index('all_type_group').T.to_dict()


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [16]:
both_file_Q1.head(2)

,Date_Saturday,actual_file,Date_Monday,forecast_file
0,2018-04-07,/home/jian/Projects/Big_Lots/Weather/Json_data...,2018-04-03,/home/jian/Projects/Big_Lots/Weather/Json_data...
1,2018-02-24,/home/jian/Projects/Big_Lots/Weather/Json_data...,2018-02-20,/home/jian/Projects/Big_Lots/Weather/Json_data...


In [17]:
output_validation=pd.DataFrame()

index_num=1
for i in range(len(both_file_Q1)):
    json_actual=json.load(open(both_file_Q1['actual_file'][i],"r"))
    json_forecast=json.load(open(both_file_Q1['forecast_file'][i],"r"))
    date=both_file_Q1['Date_Saturday'][i]
    for zip_cd in inclusion_stores['zip_cd'].unique().tolist():

        if (zip_cd in list(json_forecast.keys())) & (zip_cd in list(json_actual.keys())):
            forecast_time=datetime.datetime.fromtimestamp(json_forecast[zip_cd]['list'][31]['dt']) #3:00 pm
            actual_time=datetime.datetime.fromtimestamp(json_actual[zip_cd]['dt']) 
            
            weather_forecast_list_2pm=json_forecast[zip_cd]['list'][31]['weather']
            weather_forecast_list_11am=json_forecast[zip_cd]['list'][30]['weather']
            weather_forecast_list_8am=json_forecast[zip_cd]['list'][29]['weather']
            weather_actual_list=json_actual[zip_cd]['weather']
            all_forecast_group_value_zip=[]
            #all_forecast_desc_value_zip=[]
            all_actual_group_value_zip=[]
            #all_actual_desc_value_zip=[]
            
            for j in range(len(weather_forecast_list_2pm)):
                weather_forecast_group=weather_forecast_list_2pm[j]['main']
                weather_forecast_desc=weather_forecast_list_2pm[j]['description']
                all_forecast_group_value_zip=list(set(all_forecast_group_value_zip+[weather_forecast_group]))
                #all_forecast_desc_value_zip=list(set(all_forecast_desc_value_zip+[weather_forecast_desc]))

            for j in range(len(weather_forecast_list_11am)):
                weather_forecast_group=weather_forecast_list_11am[j]['main']
                weather_forecast_desc=weather_forecast_list_11am[j]['description']
                all_forecast_group_value_zip=list(set(all_forecast_group_value_zip+[weather_forecast_group]))
                #all_forecast_desc_value_zip=list(set(all_forecast_desc_value_zip+[weather_forecast_desc]))
                
            for j in range(len(weather_forecast_list_8am)):
                weather_forecast_group=weather_forecast_list_8am[j]['main']
                weather_forecast_desc=weather_forecast_list_8am[j]['description']
                all_forecast_group_value_zip=list(set(all_forecast_group_value_zip+[weather_forecast_group]))
                #all_forecast_desc_value_zip=list(set(all_forecast_desc_value_zip+[weather_forecast_desc]))
                
            for j in range(len(weather_actual_list)):    
                weather_actual_group=weather_actual_list[j]['main']
                weather_actual_desc=weather_actual_list[j]['description']
                all_actual_group_value_zip=list(set(all_actual_group_value_zip+[weather_actual_group]))
                #all_actual_desc_value_zip=list(set(all_actual_desc_value_zip+[weather_actual_desc]))
            
            all_forecast_group_severity_zip=[]
            all_actual_group_severity_zip=[]
            
            selected_havest_forecast_group_value_zip=np.nan
            selected_havest_actual_group_value_zip=np.nan
            
            
            for k in range(len(all_forecast_group_value_zip)):
                forecast_severity_zip=group_weight_dict[all_forecast_group_value_zip[k]]['Severity']
                all_forecast_group_severity_zip=list(set(all_forecast_group_severity_zip+[forecast_severity_zip]))
                if k==0:
                    selected_havest_forecast_group_value_zip=all_forecast_group_value_zip[0]
                else:
                    if group_weight_dict[all_forecast_group_value_zip[k]]['Severity']>group_weight_dict[selected_havest_forecast_group_value_zip]['Severity']:
                        selected_havest_forecast_group_value_zip=all_forecast_group_value_zip[k]
                    
                
                
            for k in range(len(all_actual_group_value_zip)):
                actual_severity_zip=group_weight_dict[all_actual_group_value_zip[k]]['Severity']
                all_actual_group_severity_zip=list(set(all_actual_group_severity_zip+[actual_severity_zip]))
                if k==0:
                    selected_havest_actual_group_value_zip=all_actual_group_value_zip[0]
                else:
                    if group_weight_dict[all_actual_group_value_zip[k]]['Severity']>group_weight_dict[selected_havest_actual_group_value_zip]['Severity']:
                        selected_havest_actual_group_value_zip=all_actual_group_value_zip[k]
                        
            
            forecast_max_severity_zip=max(all_forecast_group_severity_zip)
            actual_max_severity_zip=max(all_actual_group_severity_zip)
            
            df_app=pd.DataFrame({"zip_cd":zip_cd,"Saturday":date,"Forecast_Time":forecast_time,"Actual_Time":actual_time,
                                 "Forecast_Severity":forecast_max_severity_zip,"Actual_Severity":actual_max_severity_zip,
                                "Forecast_Weather_Type":selected_havest_forecast_group_value_zip,"Actual_Weather_Type":selected_havest_actual_group_value_zip},index=[index_num])
            index_num=index_num+1
            output_validation=output_validation.append(df_app)
        else:
            print(zip_cd,both_file_Q1['Date_Saturday'][i])
            
output_validation=output_validation[['zip_cd','Saturday','Forecast_Time','Actual_Time','Forecast_Weather_Type','Actual_Weather_Type','Forecast_Severity','Actual_Severity']]

47546 2018-02-24
30106 2018-02-24
37620 2018-02-24
76117 2018-02-24
30458 2018-02-24
31419 2018-02-24
44511 2018-02-24
34112 2018-02-24
27407 2018-02-24
94509 2018-02-24
92553 2018-02-24
89431 2018-02-24
95051 2018-02-24
76063 2018-02-24
76116 2018-02-24
39402 2018-02-24
27534 2018-02-24
72015 2018-02-24
33813 2018-02-24
13905 2018-02-24
35806 2018-04-28


In [18]:
output_validation['Diff']=output_validation['Actual_Severity']-output_validation['Forecast_Severity']
writer=pd.ExcelWriter(writer_floder+"BL_2018 Q1 Forecast 4 days validation Tue Sat_3_Pred_Points_JL_"+today_str+".xlsx",engine="xlsxwriter")
output_validation.to_excel(writer,"all_10_weeks",index=False)
for Saturday,group in output_validation.groupby(['Saturday']):
    group.to_excel(writer,str(Saturday),index=False)
writer.save()